In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, Conv2D, BatchNormalization, ReLU
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2  # Import L2 regularizer
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import os
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd

In [2]:
# Function to preprocess spectrograms
def preprocess_spectrogram(spectrogram, target_size):
    # Normalize
    spectrogram = spectrogram / np.max(spectrogram)
    # Resize to match MobileNet input
    spectrogram = tf.image.resize(spectrogram, target_size)
    # Convert to 3 channels (stack the same data for all channels)
    spectrogram = tf.image.grayscale_to_rgb(spectrogram)
    return spectrogram.numpy()

# Data augmentation for spectrograms
def augment_spectrogram(spectrogram):
    # Time shifting
    shift = random.randint(-10, 10)
    spectrogram = np.roll(spectrogram, shift, axis=1)

    # Frequency masking
    freq_mask = random.randint(0, 10)
    spectrogram[:, freq_mask:freq_mask + 10, :] = 0

    # Time masking
    time_mask = random.randint(0, 10)
    spectrogram[time_mask:time_mask + 10, :, :] = 0

    return spectrogram


In [3]:
initial_learning_rate = 1e-4  # Start with a higher learning rate
decay_steps = 1000            # Number of steps before applying decay
decay_rate = 0.9              # Rate of decay
learning_rate_schedule = ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True  # Decay happens in discrete intervals
)

In [4]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import tensorflow as tf

def create_regression_model(input_shape, num_outputs):
    '''
    # Define Exponential Decay for the learning rate
    initial_learning_rate = 1e-4
    decay_steps = 1000
    decay_rate = 0.9
    learning_rate_schedule = ExponentialDecay(
        initial_learning_rate=initial_learning_rate,
        decay_steps=decay_steps,
        decay_rate=decay_rate,
        staircase=True
    )
'''
    # Load MobileNetV2 with pre-trained weights from ImageNet
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

    # Fine-tune MobileNetV2
    base_model.trainable = True  # Allow all layers to be trainable initially

    # Freeze all layers except the last 3
    for layer in base_model.layers[:-10]:
        layer.trainable = False

    # Add custom layers with L2 regularization
    model = Sequential([
        base_model,
        Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.009)),
        BatchNormalization(),
        ReLU(),
        Flatten(),
        Dense(128, activation='relu', kernel_regularizer=l2(0.009)),
        Dropout(0.5),
        Dense(num_outputs, kernel_regularizer=l2(0.009))  # Output layer for regression
    ])

    # Compile the model with the learning rate schedule
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate_schedule),
        loss='mean_squared_error',  # Loss function for regression
        metrics=['mean_absolute_error']  # Metrics for regression
    )

    return model


In [5]:
# Input shape for spectrograms
input_shape = (128, 128, 3)
num_outputs = 5

# Create the model for regression
model = create_regression_model(input_shape, num_outputs)

# Compile the model
#model.compile(
    #optimizer=tf.keras.optimizers.Adam(learning_rate=ExponentialDecay(
     #   initial_learning_rate=1e-4,
      #  decay_steps=1000,
       # decay_rate=0.9,
        #staircase=True
    #)),
#    loss='mean_squared_error',  # Loss function for regression
#    metrics=['mean_absolute_error']  # Metrics for regression
#)

# Display model summary
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_128            │ (None, 4, 4, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 4, 4, 128)      │     1,474,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 4, 4, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,996,101 (15.24 MB)

 Trainable params: 2,470,341 (9.42 MB)

 Non-trainable params: 1,525,760 (5.82 MB)

In [6]:
'''import os
import time
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load_data(image_folder, txt_folder, target_size):
    start_time = time.time()
    images = []
    labels = []

    for txt_file in os.listdir(txt_folder):
        if txt_file.endswith('.txt'):
            # Read and parse the label as a list of floats
            with open(os.path.join(txt_folder, txt_file), 'r') as file:
                label = list(map(float, file.read().strip().split()))  # Convert space-separated floats to a list

            # Load the corresponding image
            image_name = os.path.splitext(txt_file)[0] + '.jpg'  # Assuming spectrograms are in .jpg format
            image_path = os.path.join(image_folder, image_name)
            if os.path.exists(image_path):
                image = load_img(image_path, target_size=target_size)
                image = img_to_array(image)
                images.append(image)
                labels.append(label)

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time taken to load the dataset: {elapsed_time:.2f} seconds")

    return np.array(images), np.array(labels)'''

'import os\nimport time\nimport numpy as np\nfrom tensorflow.keras.preprocessing.image import load_img, img_to_array\n\ndef load_data(image_folder, txt_folder, target_size):\n    start_time = time.time()\n    images = []\n    labels = []\n\n    for txt_file in os.listdir(txt_folder):\n        if txt_file.endswith(\'.txt\'):\n            # Read and parse the label as a list of floats\n            with open(os.path.join(txt_folder, txt_file), \'r\') as file:\n                label = list(map(float, file.read().strip().split()))  # Convert space-separated floats to a list\n\n            # Load the corresponding image\n            image_name = os.path.splitext(txt_file)[0] + \'.jpg\'  # Assuming spectrograms are in .jpg format\n            image_path = os.path.join(image_folder, image_name)\n            if os.path.exists(image_path):\n                image = load_img(image_path, target_size=target_size)\n                image = img_to_array(image)\n                images.append(image)\n   

In [7]:
'''import os
import time
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import UnidentifiedImageError

def load_data(image_folder, txt_folder, target_size):
    start_time = time.time()
    images = []
    labels = []
    processed_files = 0  # Counter for processed files

    for txt_file in os.listdir(txt_folder):
        if txt_file.endswith('.txt'):
            # Read and parse the label as a list of floats
            with open(os.path.join(txt_folder, txt_file), 'r') as file:
                label = list(map(float, file.read().strip().split()))  # Convert space-separated floats to a list

            # Load the corresponding image
            image_name = os.path.splitext(txt_file)[0] + '.jpg'  # Assuming spectrograms are in .jpg format
            image_path = os.path.join(image_folder, image_name)

            if os.path.exists(image_path):
                try:
                    image = load_img(image_path, target_size=target_size)
                    image = img_to_array(image)
                    images.append(image)
                    labels.append(label)
                    processed_files += 1
                except UnidentifiedImageError:
                    print(f"Warning: {image_path} could not be identified as an image and was skipped.")
                except Exception as e:
                    print(f"Warning: An error occurred with {image_path} - {e}")
            else:
                print(f"Warning: Image {image_path} does not exist.")
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Total processed files: {processed_files}")
    print(f"Time taken to load the dataset: {elapsed_time:.2f} seconds")
    return np.array(images), np.array(labels)'''

'import os\nimport time\nimport numpy as np\nfrom tensorflow.keras.preprocessing.image import load_img, img_to_array\nfrom PIL import UnidentifiedImageError\n\ndef load_data(image_folder, txt_folder, target_size):\n    start_time = time.time()\n    images = []\n    labels = []\n    processed_files = 0  # Counter for processed files\n\n    for txt_file in os.listdir(txt_folder):\n        if txt_file.endswith(\'.txt\'):\n            # Read and parse the label as a list of floats\n            with open(os.path.join(txt_folder, txt_file), \'r\') as file:\n                label = list(map(float, file.read().strip().split()))  # Convert space-separated floats to a list\n\n            # Load the corresponding image\n            image_name = os.path.splitext(txt_file)[0] + \'.jpg\'  # Assuming spectrograms are in .jpg format\n            image_path = os.path.join(image_folder, image_name)\n\n            if os.path.exists(image_path):\n                try:\n                    image = load_img(

In [8]:
'''import os
import time
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import UnidentifiedImageError

def load_data(image_folder, txt_folder, target_size, fraction=1.0):
    start_time = time.time()
    images = []
    labels = []
    processed_files = 0  # Counter for processed files

    # List all text files in the label folder
    txt_files = [f for f in os.listdir(txt_folder) if f.endswith('.txt')]

    # Randomly select a fraction of the text files
    sample_size = int(fraction * len(txt_files))
    np.random.seed(42)  # For reproducibility
    selected_files = np.random.choice(txt_files, size=sample_size, replace=False)

    for txt_file in selected_files:
        # Read and parse the label as a list of floats
        with open(os.path.join(txt_folder, txt_file), 'r') as file:
            label_values = file.read().strip().split()
            # Depending on your requirements, select the desired label value
            desired_index = 1  # Adjust this index as needed
            label = float(label_values[desired_index])

        # Load the corresponding image
        image_name = os.path.splitext(txt_file)[0] + '.jpg'  # Assuming spectrograms are in .jpg format
        image_path = os.path.join(image_folder, image_name)

        if os.path.exists(image_path):
            try:
                image = load_img(image_path, target_size=target_size)
                image = img_to_array(image)
                images.append(image)
                labels.append(label)
                processed_files += 1
            except UnidentifiedImageError:
                print(f"Warning: {image_path} could not be identified as an image and was skipped.")
            except Exception as e:
                print(f"Warning: An error occurred with {image_path} - {e}")
        else:
            print(f"Warning: Image {image_path} does not exist.")

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Total processed files: {processed_files}")
    print(f"Time taken to load the dataset: {elapsed_time:.2f} seconds")
    return np.array(images), np.array(labels)'''

'import os\nimport time\nimport numpy as np\nfrom tensorflow.keras.preprocessing.image import load_img, img_to_array\nfrom PIL import UnidentifiedImageError\n\ndef load_data(image_folder, txt_folder, target_size, fraction=1.0):\n    start_time = time.time()\n    images = []\n    labels = []\n    processed_files = 0  # Counter for processed files\n\n    # List all text files in the label folder\n    txt_files = [f for f in os.listdir(txt_folder) if f.endswith(\'.txt\')]\n\n    # Randomly select a fraction of the text files\n    sample_size = int(fraction * len(txt_files))\n    np.random.seed(42)  # For reproducibility\n    selected_files = np.random.choice(txt_files, size=sample_size, replace=False)\n\n    for txt_file in selected_files:\n        # Read and parse the label as a list of floats\n        with open(os.path.join(txt_folder, txt_file), \'r\') as file:\n            label_values = file.read().strip().split()\n            # Depending on your requirements, select the desired labe

In [9]:
'''# Paths to dataset
image_folder = '/Users/goutham/Updated/reg_48k_train_images'  # Replace with your spectrogram images path
txt_folder = '/Users/goutham/Updated/reg_48k_train_labels'    # Replace with your labels path

# Load only 50% of the data
X, y = load_data(image_folder, txt_folder, (128, 128), fraction=0.5)

# Proceed with train-test split for regression
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Create data generators
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255  # Normalize image pixel values to [0, 1]
)
val_datagen = ImageDataGenerator(
    rescale=1./255  # Normalize image pixel values to [0, 1]
)

# Generators for training and validation
train_generator = train_datagen.flow(X_train, y_train, batch_size=64)
val_generator = val_datagen.flow(X_val, y_val, batch_size=64)'''

"# Paths to dataset\nimage_folder = '/Users/goutham/Updated/reg_48k_train_images'  # Replace with your spectrogram images path\ntxt_folder = '/Users/goutham/Updated/reg_48k_train_labels'    # Replace with your labels path\n\n# Load only 50% of the data\nX, y = load_data(image_folder, txt_folder, (128, 128), fraction=0.5)\n\n# Proceed with train-test split for regression\nfrom sklearn.model_selection import train_test_split\nX_train, X_val, y_train, y_val = train_test_split(\n    X, y, test_size=0.2, random_state=42\n)\n\n# Create data generators\nfrom tensorflow.keras.preprocessing.image import ImageDataGenerator\n\ntrain_datagen = ImageDataGenerator(\n    rescale=1./255  # Normalize image pixel values to [0, 1]\n)\nval_datagen = ImageDataGenerator(\n    rescale=1./255  # Normalize image pixel values to [0, 1]\n)\n\n# Generators for training and validation\ntrain_generator = train_datagen.flow(X_train, y_train, batch_size=64)\nval_generator = val_datagen.flow(X_val, y_val, batch_size=6

In [10]:
#print(len(X_train))

In [11]:
import os
import time
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import UnidentifiedImageError

def load_data(image_folder, txt_folder, target_size, fraction=1.0):
    start_time = time.time()
    images = []
    labels = []
    processed_files = 0  # Counter for processed files

    # List all text files in the label folder
    txt_files = [f for f in os.listdir(txt_folder) if f.endswith('.txt')]

    # Randomly select a fraction of the text files
    sample_size = int(fraction * len(txt_files))
    np.random.seed(42)  # For reproducibility
    selected_files = np.random.choice(txt_files, size=sample_size, replace=False)

    for txt_file in selected_files:
        # Read and parse the label as a list of floats
        with open(os.path.join(txt_folder, txt_file), 'r') as file:
            label_values = file.read().strip().split()
            # Convert all label values to floats
            label = [float(value) for value in label_values]

        # Load the corresponding image
        image_name = os.path.splitext(txt_file)[0] + '.jpg'  # Assuming spectrograms are in .jpg format
        image_path = os.path.join(image_folder, image_name)

        if os.path.exists(image_path):
            try:
                image = load_img(image_path, target_size=target_size)
                image = img_to_array(image)
                images.append(image)
                labels.append(label)
                processed_files += 1
            except UnidentifiedImageError:
                print(f"Warning: {image_path} could not be identified as an image and was skipped.")
            except Exception as e:
                print(f"Warning: An error occurred with {image_path} - {e}")
        else:
            print(f"Warning: Image {image_path} does not exist.")

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Total processed files: {processed_files}")
    print(f"Time taken to load the dataset: {elapsed_time:.2f} seconds")
    return np.array(images), np.array(labels)

'''def load_data(image_folder, txt_folder, target_size):
    start_time = time.time()
    images = []
    labels = []
    
    txt_files = [f for f in os.listdir(txt_folder) if f.endswith('.txt')]

    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(load_single_data, txt_file, image_folder, txt_folder, target_size) for txt_file in txt_files]
        for future in futures:
            result = future.result()
            if result is not None:
                image, label = result
                images.append(image)
                labels.append(label)

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time taken to load the dataset: {elapsed_time:.2f} seconds")

    return np.array(images), np.array(labels)'''

'def load_data(image_folder, txt_folder, target_size):\n    start_time = time.time()\n    images = []\n    labels = []\n    \n    txt_files = [f for f in os.listdir(txt_folder) if f.endswith(\'.txt\')]\n\n    with ThreadPoolExecutor() as executor:\n        futures = [executor.submit(load_single_data, txt_file, image_folder, txt_folder, target_size) for txt_file in txt_files]\n        for future in futures:\n            result = future.result()\n            if result is not None:\n                image, label = result\n                images.append(image)\n                labels.append(label)\n\n    end_time = time.time()\n    elapsed_time = end_time - start_time\n    print(f"Time taken to load the dataset: {elapsed_time:.2f} seconds")\n\n    return np.array(images), np.array(labels)'

In [12]:
# Paths to dataset
image_folder = '/Users/goutham/Updated/reg_48k_train_images'  # Replace with your spectrogram images path
txt_folder = '/Users/goutham/Updated/reg_48k_train_labels'    # Replace with your labels path

# Load only 50% of the data
X, y = load_data(image_folder, txt_folder, (128, 128), fraction=0.5)

# Proceed with train-test split for regression
# Split the data into training and testing sets (70% training, 30% testing)
from sklearn.model_selection import train_test_split

# Split data into 70% training and 30% testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)
# Further split training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

# Create data generators
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(
    rescale=1./255  # Normalize image pixel values to [0, 1]
)
val_datagen = ImageDataGenerator(
    rescale=1./255
)

# Generators for training and validation
train_generator = train_datagen.flow(X_train, y_train, batch_size=64)
test_generator = test_datagen.flow(X_test, y_test, batch_size=64)
val_generator = val_datagen.flow(X_val, y_val, batch_size=64)

Total processed files: 24000
Time taken to load the dataset: 34.94 seconds


In [13]:
X_train.shape


(13440, 128, 128, 3)

In [14]:
#print(y_test[:10])

In [15]:
y_train.shape

(13440, 5)

In [16]:
# Callbacks for training
callbacks = [
    ModelCheckpoint('best_model_mobilenetv2_l2_regression.keras', save_best_only=True, monitor='val_loss'),
    EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True),  # Increased patience slightly for regression
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1)  # Increased patience for LR reduction
]


In [17]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    callbacks=callbacks
)

import os

# Define the directory to save the model
save_dir = '/Users/goutham/Updated'  # You can replace this with your desired directory path

# Create the directory if it doesn't exist
os.makedirs(save_dir, exist_ok=True)

# Full path to save the model
save_path = os.path.join(save_dir, 'mobilenetv2_l2_REGRESSION.keras')

# Save the trained model
model.save(save_path)

print(f"Model saved to: {save_path}")


Epoch 1/50


/Users/goutham/miniconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


210/210 ━━━━━━━━━━━━━━━━━━━━ 67s 301ms/step - loss: 4.8177 - mean_absolute_error: 0.5091 - val_loss: 3.8303 - val_mean_absolute_error: 0.1791 - learning_rate: 1.0000e-04
Epoch 2/50
210/210 ━━━━━━━━━━━━━━━━━━━━ 61s 288ms/step - loss: 3.6585 - mean_absolute_error: 0.2129 - val_loss: 3.0259 - val_mean_absolute_error: 0.1573 - learning_rate: 1.0000e-04
Epoch 3/50
210/210 ━━━━━━━━━━━━━━━━━━━━ 62s 292ms/step - loss: 2.8467 - mean_absolute_error: 0.1812 - val_loss: 2.2708 - val_mean_absolute_error: 0.1518 - learning_rate: 1.0000e-04
Epoch 4/50
210/210 ━━━━━━━━━━━━━━━━━━━━ 62s 293ms/step - loss: 2.1096 - mean_absolute_error: 0.1618 - val_loss: 1.6247 - val_mean_absolute_error: 0.1205 - learning_rate: 1.0000e-04
Epoch 5/50
210/210 ━━━━━━━━━━━━━━━━━━━━ 64s 300ms/step - loss: 1.5037 - mean_absolute_error: 0.1445 - val_loss: 1.1411 - val_mean_absolute_error: 0.1137 - learning_rate: 9.0000e-05
Epoch 6/50
210/210 ━━━━━━━━━━━━━━━━━━━━ 72s 339ms/step - loss: 1.0570 - mean_absolute_error: 0.1313 - val_

In [18]:
# Normalize the test data
X_test = X_test / 255.0  # Since we used rescale=1./255 in the generators

# Make predictions on the test set
y_pred = model.predict(X_test)

# Compute regression metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"Test Mean Absolute Error (MAE): {mae:.4f}")
print(f"Test Mean Squared Error (MSE): {mse:.4f}")

225/225 ━━━━━━━━━━━━━━━━━━━━ 23s 87ms/step
Test Mean Absolute Error (MAE): 0.0338
Test Mean Squared Error (MSE): 0.0044


In [ ]:
# Load test data
test_image_folder = '/Users/goutham/Updated/reg_test_images'  # Replace with your test spectrogram images path
test_txt_folder = '/Users/goutham/Updated/reg_test_labels'  # Replace with your test labels path
X_test, y_test = load_data(test_image_folder, test_txt_folder, (128, 128))

Total processed files: 15999
Time taken to load the dataset: 26.19 seconds


In [ ]:
X_train.shape

(19200, 128, 128, 3)

In [ ]:
test_predictions = model.predict(X_test)


500/500 ━━━━━━━━━━━━━━━━━━━━ 45s 85ms/step


In [ ]:
metrics_list = ['mean_absolute_error', 'mape', 'r2_keras']


In [21]:
# Compute metrics
mae = mean_absolute_error(y_test, test_predictions)


r2 = r2_score(y_test, test_predictions)

print(mae)


NameError: name 'test_predictions' is not defined

In [ ]:
mse = mean_squared_error(y_test, test_predictions)


In [ ]:
mse

np.float64(0.028882897693151978)

In [19]:
print(y_test[:100])

[[0.      0.55078 0.5     0.13281 0.99609]
 [0.      0.61719 0.5     0.12891 0.99609]
 [0.      0.45703 0.5     0.13281 0.99609]
 [0.      0.39062 0.5     0.13281 0.99609]
 [0.      0.39453 0.5     0.13281 0.99609]
 [0.      0.66211 0.5     0.12891 0.99609]
 [0.      0.28125 0.5     0.12891 0.99609]
 [0.      0.52539 0.5     0.13281 0.99609]
 [0.      0.69531 0.5     0.13281 0.99609]
 [0.      0.53711 0.5     0.13281 0.99609]
 [0.      0.40234 0.5     0.12891 0.99609]
 [0.      0.4375  0.5     0.12891 0.99609]
 [0.      0.6543  0.5     0.13281 0.99609]
 [0.      0.58398 0.5     0.12891 0.99609]
 [0.      0.56055 0.5     0.13281 0.99609]
 [0.      0.30859 0.5     0.12891 0.99609]
 [0.      0.69727 0.5     0.13281 0.99609]
 [0.      0.67969 0.5     0.13281 0.99609]
 [0.      0.71094 0.5     0.13281 0.99609]
 [0.      0.375   0.5     0.12891 0.99609]
 [0.      0.59961 0.5     0.12891 0.99609]
 [0.      0.5957  0.5     0.13281 0.99609]
 [0.      0.31445 0.5     0.13281 0.99609]
 [0.      0

In [20]:
print(y_pred[:100])

[[ 8.3315303e-05  4.9030885e-01  5.1843393e-01  1.3186795e-01
   1.0663934e+00]
 [ 1.1448516e-05  4.7389084e-01  5.0392431e-01  1.3155796e-01
   1.0112556e+00]
 [-3.7586200e-05  4.7454661e-01  4.9398410e-01  1.3136029e-01
   9.7341895e-01]
 [-9.6763251e-06  4.1680637e-01  4.9938130e-01  1.3140228e-01
   9.9416387e-01]
 [ 1.2390927e-04  4.8062688e-01  5.2661598e-01  1.3201672e-01
   1.0975852e+00]
 [ 6.2833715e-05  4.7228980e-01  5.1427823e-01  1.3175875e-01
   1.0506217e+00]
 [-5.9846789e-06  4.8243815e-01  5.0036675e-01  1.3149843e-01
   9.9772727e-01]
 [ 8.7239547e-05  4.9170211e-01  5.1925820e-01  1.3188833e-01
   1.0696275e+00]
 [ 9.6430944e-05  5.1238608e-01  5.2105331e-01  1.3195467e-01
   1.0763146e+00]
 [ 2.3764034e-05  4.7409111e-01  5.0637662e-01  1.3160408e-01
   1.0205917e+00]
 [ 1.0692747e-06  4.7678927e-01  5.0180078e-01  1.3151804e-01
   1.0032160e+00]
 [ 5.4810836e-05  4.7375792e-01  5.1270366e-01  1.3173190e-01
   1.0446509e+00]
 [-2.8651673e-05  4.4384813e-01  4.95823